In [1]:
import numpy as np
import utilities as ut
import gizmo_analysis as gizmo
import halo_analysis as halo

In [2]:
lmc_metadata = {
    'm12f': {
        'snapshot': 463,
        'tree.idx': 221716
    },
    
    'm12i': {
        'snapshot': 356,
        'tree.idx': 422845
    },
    
    'm12w': {
        'snapshot': 359,
        'tree.idx': 403617
    },
    
    'm12m': {
        'snapshot': 290,
        'tree.idx': 1821315
    },
}

import pandas as pd

lmc_metadata_df = pd.DataFrame(lmc_metadata)
lmc_metadata_df.to_hdf('lmc_metadata.hdf5', 'w')

In [12]:
def trackHaloPosition(sim_dir, ref_idx):
    
    df = {
        'fs': [],
        'fc': [],
        'bs': [],
        'bc': [],
    }
    
    halt = halo.io.IO.read_tree(simulation_directory=sim_dir)
    
    tidx = ref_idx
    
    # Forward tracking
    while tidx > 0:
        df['fs'].append(halt['snapshot'][tidx])
        df['fc'].append(halt['position'][tidx])
        tidx = halt['descendant.index'][tidx]
    
    # Backward tracking
    
    tidx = ref_idx
    
    while tidx > 0:
        df['bs'].append(halt['snapshot'][tidx])
        df['bc'].append(halt['position'][tidx])
        tidx = halt['progenitor.main.index'][tidx]
        
    df['bs'] = np.flip(df['bs'])
    df['bc'] = np.flip(df['bc'])

    dat = {
        'snapshot': np.append(df['bs'], df['fs']),
        'position': np.vstack([df['bc'], df['fc']])
    }
        
    return dat